In [2]:
import sys
#sys.path = [p for p in sys.path if p.find('/opt/apps/software/') == -1]

from utils.constants import UNIVARIATE_ARCHIVE_NAMES as ARCHIVE_NAMES
from utils.constants import UNIVARIATE_DATASET_NAMES as DATASET_NAMES

from utils.utils import read_all_datasets
from utils.utils import transform_labels
from utils.utils import create_directory
from utils.utils import run_length_xps
from utils.utils import generate_results_csv

import utils

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import numpy as np
import sklearn

# import keras
# keras.backend.tensorflow_backend._get_available_gpus()

In [3]:
sys.path

['/home/leonardo.barbosa/projects/voltametry/InceptionTime',
 '',
 '/opt/apps/software/TensorFlow/2.2.0-fosscuda-2019b-Python-3.7.4/lib/python3.7/site-packages',
 '/opt/apps/software/h5py/2.10.0-fosscuda-2019b-Python-3.7.4/lib/python3.7/site-packages',
 '/opt/apps/software/SciPy-bundle/2019.10-fosscuda-2019b-Python-3.7.4/lib/python3.7/site-packages',
 '/opt/apps/software/Python/3.7.4-GCCcore-8.3.0/easybuild/python',
 '/opt/apps/software/h5py/2.10.0-fosscuda-2019b-Python-3.7.4/lib/python3.7/site-packages/h5py-2.10.0-py3.7-linux-x86_64.egg',
 '/home/leonardo.barbosa/.conda/envs/inception/lib/python37.zip',
 '/home/leonardo.barbosa/.conda/envs/inception/lib/python3.7',
 '/home/leonardo.barbosa/.conda/envs/inception/lib/python3.7/lib-dynload',
 '/home/leonardo.barbosa/.conda/envs/inception/lib/python3.7/site-packages',
 '/home/leonardo.barbosa/.conda/envs/inception/lib/python3.7/site-packages/glmnet_python-0.2.0-py3.7.egg',
 '/home/leonardo.barbosa/.conda/envs/inception/lib/python3.7/site-

In [4]:

def prepare_data():
    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]
    x_test = datasets_dict[dataset_name][2]
    y_test = datasets_dict[dataset_name][3]

    nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))

    # make the min to zero of labels
    y_train, y_test = transform_labels(y_train, y_test)

    # save orignal y because later we will use binary
    y_true = y_test.astype(np.int64)
    y_true_train = y_train.astype(np.int64)
    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder()
    enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
    y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
    y_test = enc.transform(y_test.reshape(-1, 1)).toarray()

    if len(x_train.shape) == 2:  # if univariate
        # add a dimension to make it multivariate with one dimension
        x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

    return x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc


def fit_classifier():
    input_shape = x_train.shape[1:]

    classifier = create_classifier(classifier_name, input_shape, nb_classes,
                                   output_directory) # , verbose=1

    classifier.fit(x_train, y_train, x_test, y_test, y_true)


def create_classifier(classifier_name, input_shape, nb_classes, output_directory,
                      verbose=False, build=True):
    if classifier_name == 'nne':
        from classifiers import nne
        return nne.Classifier_NNE(output_directory, input_shape,
                                  nb_classes, verbose)
    if classifier_name == 'inception':
        from classifiers import inception
        return inception.Classifier_INCEPTION(output_directory, input_shape, nb_classes, verbose,
                                              build=build)


def get_xp_val(xp):
    if xp == 'batch_size':
        xp_arr = [16, 32, 128]
    elif xp == 'use_bottleneck':
        xp_arr = [False]
    elif xp == 'use_residual':
        xp_arr = [False]
    elif xp == 'nb_filters':
        xp_arr = [16, 64]
    elif xp == 'depth':
        xp_arr = [3, 9]
    elif xp == 'kernel_size':
        xp_arr = [8, 64]
    else:
        raise Exception('wrong argument')
    return xp_arr


In [5]:
root_dir = '/mnt/nfs/proj/in-vitro/Leonardo/inception'
xps = ['use_bottleneck', 'use_residual', 'nb_filters', 'depth',
       'kernel_size', 'batch_size']
archive_name = ARCHIVE_NAMES[0]
datasets_dict = read_all_datasets(root_dir, archive_name)

In [7]:
# run nb_iter_ iterations of Inception on the whole TSC archive
classifier_name = 'inception'
nb_iter_ = 5

for iter in range(nb_iter_):
    print('\t\titer', iter)

    trr = ''
    if iter != 0:
        trr = '_itr_' + str(iter)

    tmp_output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + trr + '/'

    for dataset_name in utils.constants.dataset_names_for_archive[archive_name]:
        print('\t\t\tdataset_name: ', dataset_name)

        x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc = prepare_data()

        output_directory = tmp_output_directory + dataset_name + '/'

        temp_output_directory = create_directory(output_directory)

        if temp_output_directory is None:
            print('Already_done', tmp_output_directory, dataset_name)
            continue

        fit_classifier()

        print('\t\t\t\tDONE')

        # the creation of this directory means
        create_directory(output_directory + '/DONE')



		iter 0
			dataset_name:  Coffee
Already_done /mnt/nfs/proj/in-vitro/Leonardo/inception/results/inception/TSC/ Coffee
			dataset_name:  Meat
Already_done /mnt/nfs/proj/in-vitro/Leonardo/inception/results/inception/TSC/ Meat
		iter 1
			dataset_name:  Coffee
Already_done /mnt/nfs/proj/in-vitro/Leonardo/inception/results/inception/TSC_itr_1/ Coffee
			dataset_name:  Meat
Already_done /mnt/nfs/proj/in-vitro/Leonardo/inception/results/inception/TSC_itr_1/ Meat
		iter 2
			dataset_name:  Coffee
Already_done /mnt/nfs/proj/in-vitro/Leonardo/inception/results/inception/TSC_itr_2/ Coffee
			dataset_name:  Meat
Already_done /mnt/nfs/proj/in-vitro/Leonardo/inception/results/inception/TSC_itr_2/ Meat
		iter 3
			dataset_name:  Coffee
Already_done /mnt/nfs/proj/in-vitro/Leonardo/inception/results/inception/TSC_itr_3/ Coffee
			dataset_name:  Meat
Already_done /mnt/nfs/proj/in-vitro/Leonardo/inception/results/inception/TSC_itr_3/ Meat
		iter 4
			dataset_name:  Coffee
Already_done /mnt/nfs/proj/in-

In [8]:
# run the ensembling of these iterations of Inception
classifier_name = 'nne'

datasets_dict = read_all_datasets(root_dir, archive_name)

tmp_output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '/'

for dataset_name in utils.constants.dataset_names_for_archive[archive_name]:
    print('\t\t\tdataset_name: ', dataset_name)

    x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc = prepare_data()

    output_directory = tmp_output_directory + dataset_name + '/'

    fit_classifier()

    print('\t\t\t\tDONE')

			dataset_name:  Coffee


Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term
				DONE
			dataset_name:  Meat
				DONE


In [9]:
y_pred = np.load('/mnt/nfs/proj/in-vitro/Leonardo/inception/results/inception/TSC/Meat/y_pred.npy')

In [10]:
y_pred.shape

(60, 3)

In [11]:
dataset_name

'Meat'

In [12]:
x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc = prepare_data()

In [13]:
acc = np.mean(y_true==np.argmax(y_pred,axis=1))

In [14]:
acc

0.9333333333333333

In [20]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60, 448, 1)
(60, 3)
(60, 448, 1)
(60, 3)


In [21]:
x_train.__class__

numpy.ndarray